In [ ]:
import pygame  
import sys  # Importing the sys module for system interaction
import copy  # Importing the copy module for object duplication
import random  # Importing the random module for working with random numbers
import time  # Importing the time module for time management

pygame.init()  # Initializing Pygame

# Setting game parameters
scale = 15  # Object scale
score = 0  # Player score
level = 0  # Game level
SPEED = 10  # Snake speed

food_x = 10  # Food x-coordinate
food_y = 10  # Food y-coordinate

# Creating a game window
display = pygame.display.set_mode((500, 500))
pygame.display.set_caption("Snake Game")  # Setting window title
clock = pygame.time.Clock()  # Creating a Clock object to manage game time

# Defining colors in RGB format
background_top = (0, 0, 50)  # Top background color
background_bottom = (0, 0, 0)  # Bottom background color
snake_colour = (255, 137, 0)  # Snake color
food_colour = (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))  # Random food color
snake_head = (255, 247, 0)  # Snake head color
font_colour = (255, 255, 255)  # Font color
defeat_colour = (255, 0, 0)  # Defeat color

# Defining Snake class
class Snake:
    def __init__(self, x_start, y_start):
        self.x = x_start
        self.y = y_start
        self.w = 15
        self.h = 15
        self.x_dir = 1  # Movement direction along x-axis (1 - right, -1 - left)
        self.y_dir = 0  # Movement direction along y-axis (1 - down, -1 - up)
        self.history = [[self.x, self.y]]  # Snake movement history
        self.length = 1  # Snake length

    def reset(self):
        self.x = 500 / 2 - scale
        self.y = 500 / 2 - scale
        self.w = 15
        self.h = 15
        self.x_dir = 1
        self.y_dir = 0
        self.history = [[self.x, self.y]]
        self.length = 1

    def show(self):
        for i in range(self.length):
            if not i == 0:
                pygame.draw.rect(display, snake_colour, (self.history[i][0], self.history[i][1], self.w, self.h))
            else:
                pygame.draw.rect(display, snake_head, (self.history[i][0], self.history[i][1], self.w, self.h))

    def check_eaten(self):
        if abs(self.history[0][0] - food_x) < scale and abs(self.history[0][1] - food_y) < scale:
            return True

    def check_level(self):
        global level
        if self.length % 5 == 0:
            return True

    def grow(self):
        self.length += 1
        self.history.append(self.history[self.length - 2])

    def death(self):
        i = self.length - 1
        while i > 0:
            if abs(self.history[0][0] - self.history[i][0]) < self.w and abs(self.history[0][1] - self.history[i][1]) < self.h and self.length > 2:
                return True
            i -= 1

    def update(self):
        i = self.length - 1
        while i > 0:
            self.history[i] = copy.deepcopy(self.history[i - 1])
            i -= 1
        self.history[0][0] += self.x_dir * scale
        self.history[0][1] += self.y_dir * scale

# Defining Food class
class Food:
    def new_location(self):
        global food_x, food_y
        food_x = random.randrange(1, int(500 / scale) - 1) * scale
        food_y = random.randrange(1, int(500 / scale) - 1) * scale

    def show(self):
        pygame.draw.rect(display, food_colour, (food_x, food_y, scale, scale))

# Function to display player score
def show_score():
    font = pygame.font.SysFont(None, 20)
    text = font.render("Score: " + str(score), True, font_colour)
    display.blit(text, (scale, scale))

# Function to display game level
def show_level():
    font = pygame.font.SysFont(None, 20)
    text = font.render("Level: " + str(level), True, font_colour)
    display.blit(text, (90 - scale, scale))

# Main game loop
def gameLoop():
    global score
    global level
    global SPEED

    snake = Snake(500 / 2, 500 / 2)
    food = Food()
    food.new_location()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    pygame.quit()
                    sys.exit()
                if snake.y_dir == 0:
                    if event.key == pygame.K_UP:
                        snake.x_dir = 0
                        snake.y_dir = -1
                    if event.key == pygame.K_DOWN:
                        snake.x_dir = 0
                        snake.y_dir = 1
                if snake.x_dir == 0:
                    if event.key == pygame.K_LEFT:
                        snake.x_dir = -1
                        snake.y_dir = 0
                    if event.key == pygame.K_RIGHT:
                        snake.x_dir = 1
                        snake.y_dir = 0

        for y in range(500):
            color = (
                background_top[0] + (background_bottom[0] - background_top[0]) * y / 500,
                background_top[1] + (background_bottom[1] - background_top[1]) * y / 500,
                background_top[2] + (background_bottom[2] - background_top[2]) * y / 500
            )
            pygame.draw.line(display, color, (0, y), (500, y))

        snake.show()
        snake.update()
        food.show()
        show_score()
        show_level()

        if snake.check_eaten():
            food.new_location()
            score += random.randint(1, 5)
            snake.grow()

        if snake.check_level():
            food.new_location()
            level += 1
            SPEED += 1
            snake.grow()

        if snake.death():
            score = 0
            level = 0
            font = pygame.font.SysFont(None, 100)
            text = font.render("Game Over!", True, defeat_colour)
            display.blit(text, (50, 200))
            pygame.display.update()
            time.sleep(3)
            snake.reset()
        if snake.history[0][0] > 500: 
            snake.history[0][0] = 0  

        if snake.history[0][0] < 0:  
            snake.history[0][0] = 500 
            
        if snake.history[0][1] > 500:
            snake.history[0][1] = 0
            
        if snake.history[0][1] < 0:
            snake.history[0][1] = 500

        pygame.display.update()
        clock.tick(SPEED)

gameLoop()
